In [208]:
import pandas as pd
import numpy as np
import json
# データ可視化ライブラリ
import matplotlib.pyplot as plt
%matplotlib inline  
import seaborn as sns
import re
from tqdm import tqdm_notebook as tqdm

import lightgbm as lgb
print('lightgbm version:', lgb.__version__)

from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import sklearn
print('sklearn version:', sklearn.__version__)

import warnings
warnings.filterwarnings('ignore')

lightgbm version: 2.3.0
sklearn version: 0.22.1


In [209]:
train = pd.read_csv('input/train_data.csv')
test = pd.read_csv('input/test_data.csv')

In [210]:
# ↑のjsonファイルをnames.jsonで保存してある前提
with open("columns.json", "r", encoding="utf-8") as f:
     d = json.load(f)
        
train = train.rename(columns=d)
test = test.rename(columns=d) 

In [211]:
delete_columns = ['id', 'Prefecture', 'Municipality', 'DistrictName']
train = train.drop(delete_columns, axis=1)
test = test.drop(delete_columns, axis=1)

In [212]:
train = train.drop_duplicates()

In [213]:
data = pd.concat([train, test])

In [214]:
data['building_id'] = data.groupby(['MunicipalityCode', 'TimeToNearestStation','BuildingYear', 'Structure', 'Use', 'NearestStation', 'FloorPlan']
                              ).grouper.group_info[0]

In [215]:
data_y = data[['building_id', 'y']]

In [216]:
data_2 = data_y.groupby('building_id').mean()
data_3 = data_y.groupby('building_id').max()
data_4 = data_y.groupby('building_id').min()
data_5 = data_y.groupby('building_id').count()

data_y['mean'] = data_2['y']
data_y['max'] = data_3['y']
data_y['min'] = data_4['y']
data_y['count'] = data_5['y']
data_y['max_min'] = data_y['max'] / data_y['min']

In [217]:
data_y

,building_id,y,mean,max,min,count,max_min
0,-1,66.0,63.000000,63.0,63.0,1.0,1.000000
1,-1,19.0,75.000000,75.0,75.0,1.0,1.000000
2,472,37.0,110.000000,110.0,110.0,1.0,1.000000
3,819,18.0,130.000000,130.0,130.0,1.0,1.000000
4,-1,240.0,18.333333,20.0,16.0,3.0,1.250000
...,...,...,...,...,...,...,...
34839,-1,NaN,17.666667,26.0,10.0,6.0,2.600000
34840,-1,NaN,16.600000,26.0,9.8,8.0,2.653061
34841,-1,NaN,20.000000,20.0,20.0,1.0,1.000000
34842,-1,NaN,8.700000,8.7,8.7,1.0,1.000000


In [218]:
test_y = data_y[len(train):]

In [219]:
test_y.query('count > 3 and max_min < 1.3')

,building_id,y,mean,max,min,count,max_min
63,479,NaN,47.200000,54.0,43.0,5.0,1.255814
65,-1,NaN,19.333333,21.0,17.0,6.0,1.235294
132,-1,NaN,13.750000,15.0,13.0,4.0,1.153846
144,-1,NaN,59.500000,66.0,51.0,4.0,1.294118
169,-1,NaN,31.000000,35.0,29.0,4.0,1.206897
...,...,...,...,...,...,...,...
34712,-1,NaN,22.833333,25.0,21.0,6.0,1.190476
34721,-1,NaN,16.000000,18.0,14.0,4.0,1.285714
34752,-1,NaN,20.000000,23.0,18.0,4.0,1.277778
34769,-1,NaN,32.000000,35.0,29.0,4.0,1.206897


In [278]:
sub = pd.read_csv('sub_20204016_3.csv')

In [287]:
for i in range(len(test_y)):
    if test_y['count'][i] > 6 and test_y['max_min'][i] < 1.1:
        sub['y'][i] = np.round(test_y['mean'][i], 2)
        print(1,  np.round(test_y['mean'][i], 2))
    if sub['y'][i] < 0:
        sub['y'][i] = 0

1 71.14
1 23.9
1 25.0
1 24.0
1 26.14
1 23.0
1 43.14
1 22.14
1 23.25
1 24.14
1 21.67
1 23.0
1 28.89


In [288]:
sub.query('y < 0')

,id,y


In [289]:
sub.to_csv('sub_20204016_3_kai_4.csv', index=False)

## わかったこと
```
for i in range(len(test_y)):
    if test_y['count'][i] > 6 and test_y['max_min'][i] < 1.1:
        sub['y'][i] = np.round(test_y['mean'][i], 2)
        print(1,  np.round(test_y['mean'][i], 2))
    if sub['y'][i] < 0:
        sub['y'][i] = 0
```
カウント6以上だと精度が下がる.ただ0.1くらいの変化なのでやってもいいかも？
やるとしてもcount7以上が良さそう！

0以下を0に変えるとスコアが上がる！  
別の何かで置き換えられたらなお良さそう！